In [8]:
import pandas as pd
import numpy as np

file_name_out='ENT_CA_DM_Meds_Outpatient.txt'
file_name_in='ENT_CA_DM_Meds_Inpatient.txt'
df_out=pd.read_csv(file_name_out,delimiter='|',error_bad_lines=False, parse_dates=True)
df_in=pd.read_csv(file_name_in,delimiter='|',error_bad_lines=False, parse_dates=True)
cols=['mrn', 'enc_date', 'thera_class', 'med_name', 'generic_name', 'pharm_class', 
      'order_start_dttm', 'order_end_dttm', 'order_med_freq', 'ordering_mode']
df_out=df_out[cols]
df_in=df_in[cols]
df=df_out.append(df_in)
print ('df size:',df.shape[0])

### filter mrns to only the ones from progress notes doc
mrn_df=pd.read_csv('HNDB Progress Notes Processed.csv')
mrns_to_use=mrn_df.mrn.unique()
print ('Number of target mrns:',len(mrns_to_use))

df=df[df['mrn'].isin(mrns_to_use)]
print ('df size for only target mrns:',df.shape[0])

df size: 2000610
Number of target mrns: 171
df size for only target mrns: 207965


In [9]:
import re

def isChemoDrug(drug):
    try:
        drug_lower=drug.lower()
        drugs=['everolimus', 'hydroxyurea', 'nab-paclitaxel', 'paclitaxel', 
               'erlotinib', 'fluorouracil', 'cisplatin', 'cetuximab', 
               'docetaxel', 'carboplatin', 'nivolumab', 'pembrolizumab', 'methotrexate']
        pattern=r""
        for index,d in enumerate(drugs):
            if index==0:
                pattern=d
            else:
                pattern=pattern+'|'+d
        search=re.search(pattern,drug_lower)
        if search:
            return True
        else:
            return False
    except:
        return False

mask=df['med_name'].apply(isChemoDrug)
df=df[mask]
print ('df size with only chemo drugs:',df.shape[0])

df size with only chemo drugs: 14400


In [10]:
def get_med(med_name):
    try:
        return med_name.split(' ')[0]
    except:
        return med_name


df['enc_date']=pd.to_datetime(df['enc_date'],infer_datetime_format=True)
df['order_start_dttm']=pd.to_datetime(df['order_start_dttm'],infer_datetime_format=True)
df['order_end_dttm']=pd.to_datetime(df['order_end_dttm'],infer_datetime_format=True)
df['med']=df.med_name.apply(get_med)

### in case order_start or end date is not populated, use enc_date in that position
for index,row in df.iterrows():
    if pd.isnull(row['order_start_dttm']):
        df.at[index, 'order_start_dttm']= row['enc_date']
    if pd.isnull(row['order_end_dttm']):
        df.at[index, 'order_end_dttm']= row['enc_date']

print (df.columns)

Index(['mrn', 'enc_date', 'thera_class', 'med_name', 'generic_name',
       'pharm_class', 'order_start_dttm', 'order_end_dttm', 'order_med_freq',
       'ordering_mode', 'med'],
      dtype='object')


In [11]:
df=df.sort_values(by='enc_date')

unique_mrns=df.mrn.unique()
print ('Number of unique ptx:',len(unique_mrns))

ptx_info=[]
for mrn in unique_mrns:
    sub_df=df[df['mrn']==mrn]
    ptx_info.append(sub_df)

print (ptx_info[0])

Number of unique ptx: 155
            mrn   enc_date      thera_class  \
292551  3054505 2009-02-03  ANTINEOPLASTICS   
292550  3054505 2009-02-03  ANTINEOPLASTICS   
305796  3054505 2009-02-03  ANTINEOPLASTICS   
292635  3054505 2009-02-03  ANTINEOPLASTICS   
292636  3054505 2009-02-03  ANTINEOPLASTICS   
...         ...        ...              ...   
170837  3054505 2009-03-20  ANTINEOPLASTICS   
401201  3054505 2009-03-20  ANTINEOPLASTICS   
291793  3054505 2009-03-20  ANTINEOPLASTICS   
291794  3054505 2009-03-20  ANTINEOPLASTICS   
170933  3054505 2009-03-20  ANTINEOPLASTICS   

                                    med_name                generic_name  \
292551  HYDROXYUREA 40 MG/ML ORAL SUSPENSION                         NaN   
292550  HYDROXYUREA 40 MG/ML ORAL SUSPENSION                         NaN   
305796            HYDROXYUREA 500 MG CAPSULE  hydroxyurea 500 mg capsule   
292635            HYDROXYUREA 500 MG CAPSULE  hydroxyurea 500 mg capsule   
292636                       

In [12]:
import plotly.figure_factory as ff

def make_data(df):
    '''make the data to input into gantt chart, taking input of df '''
    data=[]
    for index,row in df.iterrows():
        Task=row['med']
        Start=row['order_start_dttm']
        End=row['order_end_dttm']
        Source=row['ordering_mode']
        if index==0:
            data=[dict(Task=Task,Start=Start,Finish=End,Source=Source)]
        else:
            data.append(dict(Task=Task,Start=Start,Finish=End,Source=Source))
    return data
def plot_gannt(df):
    colors = {'Inpatient':(0,0,1),
          'Outpatient':	(1,1,0),
          'RAD':(0, 1, 0),
             'Surgery':(0,.71,.75)}
    fig = ff.create_gantt(make_data(df),colors=colors,index_col='Source', show_colorbar=True,
                      group_tasks=True)
    fig.show()

for i in range(10):
    plot_gannt(ptx_info[i])

In [13]:
### combine all dataframes into one
df_med=''
for index,ptx in enumerate(ptx_info):
    if index==0:
        df_med=ptx
    else:
        df_med=df_med.append(ptx)
print (df_med.columns)
cols=['mrn','med','order_start_dttm','order_end_dttm','ordering_mode']
df_med=df_med[cols]
### load radiation information
df_rad=pd.read_csv('RM_radiation_cycles.csv')
# set med type to RAD for radiation
df_rad['med']='RAD'
df_rad['source']='RAD'
print (df_rad.columns)
cols=['mrn','med','start','end','source']
df_rad=df_rad[cols]
df_rad.columns=['mrn','med','order_start_dttm','order_end_dttm','ordering_mode']
df_rad['order_start_dttm']=pd.to_datetime(df_rad['order_start_dttm'],infer_datetime_format=True)
df=df_med.append(df_rad)
print (df.shape[0])
### get the surgery info- this trashes the scaling so remove if you want a prettier pic
df_surg=pd.read_csv('RM_surgeries.csv')
df_surg['order_start_dttm']=pd.to_datetime(df_surg['order_start_dttm'],infer_datetime_format=True)
df_surg['order_end_dttm']=pd.to_datetime(df_surg['order_end_dttm'],infer_datetime_format=True)
df=df.append(df_surg)
### sort by order_start_dttm
df=df.sort_values(by='order_start_dttm')
df.to_csv('examine.csv')
# convert ordering mode to string
df['ordering_mode']=df['ordering_mode'].astype(str)
# split into patient level info
ptx_info=[]
for mrn in df.mrn.unique():
    sub_df=df[df['mrn']==mrn]
    ptx_info.append(sub_df)
print (len(ptx_info))

Index(['mrn', 'enc_date', 'thera_class', 'med_name', 'generic_name',
       'pharm_class', 'order_start_dttm', 'order_end_dttm', 'order_med_freq',
       'ordering_mode', 'med'],
      dtype='object')
Index(['mrn', 'cycle', 'start', 'end', 'med', 'source'], dtype='object')
14521
191


In [14]:
for ptx in ptx_info:
    # these are all OPTIMA mrns and one person with a late resection - 2015
    mrns=[654028,3581525,3557665,3553505,3552528,2209904,1267842,3542051,3539092,3535574,3519848,3519179,3508689,3503678,3497326,349581]
    if ptx.iloc[0]['mrn'] in mrns:
        plot_gannt(ptx)

# for i in range(30):
#     plot_gannt(ptx_info[i])